In [1]:
# load model 
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch
from accelerate import init_empty_weights
from shadedLM import get_tokens, ShardedChatglm26B, WeightsLoader
model_name = "/home/scc/Downloads/AIToy/ChatGLM2_localTrain/code/models/chatglm2-6b"
# “THUDM/chatglm2-6b”

torch.__version__
N_BATCHES = 3
MAX_LENGTH = 4096

In [3]:

# weights_loader = WeightsLoader(model_name, devices=['cuda:0'])
# model = ShardedChatglm26B(model_name, weights_loader, device='cuda:0')
# str_in = "what's your name?"
# inputs = get_tokens(str_in, tokenizer)
# outputs = model(inputs)

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True
) # .half().cuda() # use FP16
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)

# 4-bit quanlite
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True, # QLoRA designed Double Quantization
    bnb_4bit_quant_type='nf4', # QLoRA designed Normal Float 4 quanlit data type
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False
)

model = AutoModel.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)


In [4]:
response, his = model.chat(tokenizer, "Whats your name?", history=[])
print(response)

In [ ]:
from IPython.display import display, Markdown, clear_output

def display_answer(query, history=[]):
    for response, history in model.stream_chat(
            tokenizer, query, history=history):
        clear_output(wait=True)
        display(Markdown(response))
    return history

display_answer("你叫什么名字？")

In [1]:
# 准备提示语
prompt = '''
一个人思考问题的方式，可以分6大层次。

第1个层次是现象层，即看到的都是纷繁芜杂的表面事物。

第2个层次是工具层，即不但看到了各种表面事物，还在表面事物中总结及归纳出了工具，通过工具更好地完成表象的工作。

第3个层次在方法层，就是在工具的基础上，能总结出解决问题的思考方式及行为方式，并解决方案化，从而能持续更高效地解决问题。

第4个层次在方法论层，就是不仅能总结出方法，而且还能将方法系统化，形成一整套解决问题为目的的体系或者系统，从而从全局层面，而非局部层面解决问题。

第5个层次是框架层，框架比方法更为抽象，是人类对底层规律进行探究及抽取、封装之后形成的模型，是解决问题的原理级存在，能普遍适用于跨行业、跨领域、跨学科的问题解决。

第6个层次是认知层，认知是人们对事物的认知层次，张一鸣说：“你对一件事情的理解，就是你的竞争力；你对一件事情的认知越深刻、越有竞争力”，认知所有竞争要素中最底层要素，认知就是你对事物最底层的规律触达能力，如广义相对论的最底层规律就一个公式E=MC2，认知要素是一个人所有竞争要素中最难建立的要素。


— 2 —
任何一个人做一件事情，都超越不了他思考问题的方式。

比如夫妻两个人想要开一家包子铺，这个包子铺也有6种开法。

第一种开法，叫做开成“小店知名”的包子铺，是一种夫妻老婆热炕头的开发，夫妻两个人早4晚10，一个人负责进货、生产，一个人负责卖货、账务，分工有序、勤勤力力，守着手艺，守着基础，不求大进，足够认真负责，能做到小店知名，也能每天晚上回家数钱；

第二种开法，叫开成“小区知名”包子铺，夫妻两个人把做包子的手艺更加工具化了，提升做包子效率，并且能复制到第二家、第三家店，统一进货，多店销售，不单单是自己做，还雇佣了人手一起来做，这种无疑回家数的钱要增加很多；

第三种开法，叫开成“城里出名”的包子铺，要开成这样的包子铺，夫妻两人就不能只有工具，必须要总结方法，什么样的包子更好吃，什么样的原料搭配、什么样的肥瘦比、什么样的制作工艺、什么样蒸煮工艺等都要有方法，方法总结得越好，包子铺的竞争力也就越强；

第四种开法，叫开成“省里知名”的包子铺，要变成一个区域性领导的品牌，意味要批量化地复制成功，这样就不仅需要方法，而是需要一整套的方法论了，就需要门店口味标准化、门店产品标准化、门店服务标准化、门店管理标准化方面形成省里的竞争力；

第五种开法，叫开成“全国知名”的包子铺，要开成这样的包子铺，就相当于开一个中国的包子“麦当劳”，就光是方法论也不够了，需要有框架思维，因为“麦当劳”的成功不仅仅是汉堡好吃，而是其背后强大的供应链，其嵌入到主流饮食文化的运作能力，其不断的品牌塑造及品牌文化输入能力，其持续的品牌与新一代消费者融合能力，需要通过框架将别人成功的经验复制到一个开创性的业务上；

第六种开法，叫开成“世界知名”的包子铺，要达成这样的高度，就不能单单就包子论包子，要从全球的餐饮连锁品牌及全球餐饮文化发展的底层逻辑，去寻找品牌全球知名的机会，比如全球主食流量入口的连锁品牌有汉堡的麦当劳、披萨的Pizza Hut、三明治的Subway，中国如何跨越文化及饮食习惯壁垒，如何入侵其他国家消费者的生活习惯，如何在其他国家人民生活场景中占有一席之地，需要深刻的知识及认知。




— 3 —
你是第几流人才？
1、第5流人才
别名：随波逐流者

所处理解层次：环境

典型思考模式：跟着现象走，跟着感觉走，经常抱怨，看到这边有问题、那边有问题


理解层次的最低层是现象及表现。处在这个理解层次的人，当问题发生的时候，他首先会把问题归结成因为环境的不好而产生的问题，而不是找问题背后深层次的原因。

比如：

业务没有做成，是因为产品不好、竞争太激烈、客户没有预算、我们去的太晚、我们运气不好等等等等，总之总是能从环境中找到问题，发生这个问题，习惯于从外部找问题……


2、第4流人才
别名：不断行动

所处理解层次：行动及应用工具

典型思考模式：勤奋及熟练


理解层次的低层的思考及行为。处在这个理解层次的人，能积极地行动，能创造及应用工具，就如早期的智人与猿人的区分一样，已经具备更强的协作能力，就如工厂的工人一样，通过行为与操作，创造好的产品。

比如：

这次销售不好，我是不是可以提炼一个更好的卖点，我是不是优化一下我的话术，我的推进节奏及推进的重点是不是可以更优化一些，我们是不是可以总结一些销售道具，下次作业就可以更成功……


3、第3流人才

别名：思考者

所处理解层次：方法

典型思考模式：不仅是行动这，还是思考者，抽象思维一旦介入，方法总比问题多！

在这个层次上，环境已变得不是最重要的，他们总觉得自己不够努力，他们也总觉得会有更好的方法。


什么是方法？就是解决问题的思考方法及行为方式构成的解决方案，意味着你能用更简单、更高效的方式解决同样的问题。

理解层次处在方法层次的人，当问题发生的时候，他们会深度思考问题，透过现象找本质，透过表象找真相，去寻找更好的方法来解决问题。

比如：

做为一家咨询公司如果获客渠道不够，就要考虑更好的营销方法，如知识圈地的方法，目前知识型直播，视频切片分发的方法，培训植入的方法，乃至于重点客户针对性开发的方式等等……
这类人有非常强大的学习能力和应用能力，能把学习到的知识，转化为可操作的方法，进而改善效率，解决问题。

如果你能走到这个层次，既有行为层的勤奋努力，又有能力层的方法套路，一般就能成为公司里的中高层了。普通的问题已经难不倒你了，你总能找到办法来解决它们。

4、第2流人才

别名：战术家

所处理解层次：一整套的方法体系

典型思考模式：如何能讲最佳实践总结成体系化的方法，从实践到方法，再从方法到实践。


什么是方法论？就是解决问题的思考方法及行为方式构成的解决方案，而方法构成的体系让系统化地复制成功成为可能。

一整套方法体系是将最佳商业实践以抽象的方式高度概括出来，从而指导未来的商业实践。

比如：

我们党的战争年代的战术原则，对商战有极强的启发意义：
一、一点两面：进攻时只攻一点，包围两面(敞开两面，甚至三面)；可以“一点一面”，也可以“一点两面”，乃至于“一点多面，三面，四面”；可以“有点无面”，可以“多点多面”，可以“点中有面，面中有点”，中心是为了集中兵力，只讲自己点上、面上兵力对比，把自己兵力集中在一点上，至于同战场上敌军兵力对比，则可以不占优势。
二、四快一慢：抓住敌人后的准备要快，前进要快，扩张战果要快，敌溃退后追击要快。总攻开始时间要慢，中心是一个“慢”字。慢的时间用以侦察地形、敌情、布署兵力、火力，进行政治鼓动工作，休息。不管情况与任务怎样，反正我要准备好了才打，没有准备好就不打，你有千条计，我有老主意。攻击准备时间要超过战斗整个时间。
三、三三制：：步兵班编为3-4个战斗小组，每组3-4人，老战士、新战士、解放战士结合配置，攻击时以小组为单位，呈三角形队形。这样进攻时队形疏散，减少伤亡。
四、三猛：猛打、猛冲、猛追。猛打、猛冲：必须拼命打，只管拼命打，第一梯队打不下来就换第二梯队打，第二梯队打不下来就换第三梯队打，照原计划硬打下去。猛追：(追击溃敌时)不管三七二十一，一下子拥上去，发扬冒险精神，不要怕情况不清楚，追就是了。以乱对乱，冒险扩张战果，死追硬追穷追。一切战术中最重要的战术就是死打，打光就打光，完蛋就完蛋。
五、三种情况三种打法：根据敌人三种不同情况(防御、退却、要退而不退)，采取三种不同的攻击方法(不打莽撞仗，要打莽撞仗，半打莽撞仗)，一种是敌人守：要完成一切准备工作再攻击。一种是敌人要退不退：如果准备好了再打，敌人会跑掉。不准备就打，又会碰不下来。这时应先将敌人围起来，围而不攻或围而小攻，用一部分和他打，拖住他，使他摆不脱，然后准备好了再大打。再一种是敌人退：那就要猛追，这时不要等命令，不要怕部队少，也不要怕情况不清楚，追就是了。
六、四组一队：以连为单位，设“四组”：火力组、突击组、爆破组、支援组合起来为“一队”：突击队。是为了解决攻城战、巷战中的队形问题。同时说明不受四个组的限制，可以灵活运用。
这个级别的人才，已经是万里挑一，解决复杂的问题，如庖丁解牛、游刃有余，一般已经能指挥一家企业，乃至于指挥一个军团作战。

5、第1流人才

别名：战略家

所处理解层次：有一整套高度抽象的框架及拥有极为深刻的原认知

典型思考模式：第一性原理


第一性原理的思想方式是用物理学的角度看待世界，也就是说一层层拨开事物表象，看到里面的本质，再从本质一层层往上走。

第一性原理就是找到问题最开始的起点，即“元起点”或者“元问题”。我们运用第一性原理，而不是比较思维去思考问题是非常重要的。我们在生活中总是倾向于比较，对别人已经做过或者正在做的事情我们也都去做，这样发展的结果只能产生细小的迭代发展。第一性原理的思想方式是用物理学的角度看待世界，也就是说一层层拨开事物表象，看到里面的本质，再从本质一层层往上走。

第一性思维即找到这个行业底层逻辑，基于底层逻辑来构建整个业务大厦。

也就是毛主席说的“全局决定性规律生发出一切战略战术”。


张一鸣说：“你对一件事情的理解，就是你的竞争力；你对一件事情的认知越深刻、越有竞争力”。

电影《教父》中，就有这样一句台词:“花三分钟看透本质的人，和花一辈子都看不清的人，注定拥有截然不同的人生。”

认知所有竞争要素中最底层要素，而这里说的认知就是你是否能看透事情的本质，抓住事物最底层的规律；认知要素最难建立，而其他的竞争要如你的团队构建、组织管理、日常运营、融资等都是可以基于你的认知则都可以构建，你的认知越深刻，你的其他要素的组合能力就越准确、也越强、越有竞争力，所以说认知是决定成败的核心要素。


框架是人们对复杂世界的底层规律的一种高度总结，一种高度抽象提炼封装而成的模型。

框架是高手从上帝手里‘偷来’的地图。

亚里士多德说““每个系统中存在一个最基本的命题，它不能被违背或删除”，这个基本命题就是事物的本质及底层规律，最优秀的人才，都是基于事物底层规律，制定战略，选择方向。

系统中的基本命题就是一个基础框架。

战略对了，就建立了全局性的优势，就朝着正确的方向进发，并不断积累品牌资产，建立自己的核心竞争优势，建立起自己的护城河。

先定大道理，再定小道理，小道理跟着大道理跑。

善谋局者，一子失着，全盘可以弥补；而谋子者，却常常顾此失彼，一着不慎，全盘皆输。


正确的决定，并不是企业家每一个小决定都是正确的，

而是企业家每一个小决定都不偏离大方向。

企业的所有小决定，都需要在企业的战略方针指引下做出。

毛主席讲：因为懂得了全局性的东西，就更会使用局部性的东西，

因为局部性的东西是隶属于全局性的。小决定是战术，大战略是方向，战术跟着战略走，大方向不偏，小决定错不到哪里去。

第一流人才是框架强的人才，是元认知强的人才。

毛主席就是这样人才的典型代表，所以完成了200年来最伟大的创业故事。


这样的人才，就可以谋篇布局、纵横捭阖，注意力总是流向机枢根本，永远提纲掣领，总是撇开细枝末节，“向大本大源处探讨”，善于抓住根本性、规律性的决定因素，逻辑推导一切战略和策略，纲举目张、执一御百，从而牢控全局。


'''

In [2]:
history = display_answer("人思考的层次分类")

NameError: name 'display_answer' is not defined

In [ ]:
# 注入文章内容后的回答
history = display_answer(prompt + "\n 根据这篇文章内容，请回答我的问题：人思考的层次分类？它们的特点是什么？")

In [ ]:
# 注入文章内容后的回答
history = display_answer(prompt + "\n 根据这篇文章内容，请回答我的问题：如何根据文中人才的分类指导自我学习？")